# 네이버 보이스피싱 관련 기사 크롤링

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

title_text=[]
date_text=[]                                                                #각 크롤링 결과 저장하기 위한 리스트 선언 
result={}

                                                    
RESULT_PATH ='./'                                                            #결과 저장할 경로


                                                                             #날짜 정제화 함수
def date_cleansing(test):
    try:
        pattern = '\d+.(\d+).(\d+).'  
        r = re.compile(pattern)
        match = r.search(test).group(0)  
        date_text.append(match)
        
    except AttributeError: 
        pattern = '\w* (\d\w*)'     
        
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)


def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)                                           #제목
            
        
                                                                                   #날짜 추출 
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)
        

        result= {"date" : date_text , "title":title_text}  
        print(page)
        
        df = pd.DataFrame(result) 
        page += 10
    
    
                                                                                 # 새로 만들 파일이름 지정
    outputFileName = '보이스피싱().xlsx'
    df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')
    
    

def main():
    
    maxpage = 400
    query = '보이스피싱' 
    sort = '0'                                                                       #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(2019.01.04):") 
    e_date = input("끝날짜 입력(2019.01.05):")   
    
    crawler(maxpage,query,sort,s_date,e_date) 
    
main()

In [11]:
df = pd.DataFrame(result)